In [ ]:
import os
os.chdir(os.path.dirname(os.getcwd()))
print(os.getcwd())

In [ ]:

from utils.depth_operations_functional import cost_volume
from tensorflow import keras as ks, float32
import tensorflow as tf

def cost_volume2(c1, search_range, name="cost_volume", dilation_rate=1,
                nbre_cuts=1):
    """Build cost volume for associating a pixel from Image1 with its corresponding pixels in Image2.
    Args:
        c1: Feature map 1
        c2: Feature map 2
        search_range: Search range (maximum displacement)
    """
    print("in", c1.shape)
    max_offset=search_range*2+1
    c1 = tf.cast(c1, tf.float16)

    strided_search_range = search_range * dilation_rate
    padded_lvl = tf.pad(c1, [[0, 0],
                             [strided_search_range, strided_search_range],
                             [strided_search_range, strided_search_range],
                             [0, 0]])
    _, h, w, c = c1.get_shape().as_list()
    c1_nchw = c1 #tf.transpose(c1, perm=[0, 3, 1, 2])
    pl_nchw = padded_lvl #tf.transpose(padded_lvl, perm=[0, 3, 1, 2])

    list_all2=tf.image.extract_patches(pl_nchw,
                                       sizes=[1, h, w, 1],
                                       strides=[1, dilation_rate, dilation_rate,1],
                                       rates=[1,1,1,1],
                                       padding="VALID")

    patches = ks.layers.Reshape(list_all2.shape[1:3] + c1.shape[1:4], dtype=tf.float16)(list_all2)
    c1_repeat = tf.expand_dims(c1,axis=1)
    c1_repeat = tf.expand_dims(c1_repeat,axis=1)
    c1_repeat = tf.tile(c1_repeat, [1,patches.shape[1], patches.shape[2],1, 1, 1])

    patches = c1_repeat*patches
    split_stack = tf.stack(tf.split(patches, num_or_size_splits=nbre_cuts, axis=-1), axis=-1)
    cost = tf.reduce_mean(split_stack, axis=-2)
    cost = tf.transpose(cost, perm=[0, 3, 4, 1, 2, 5])
    cost_vol = ks.layers.Reshape(cost.shape[1:3] + [cost.shape[3]*cost.shape[4]*cost.shape[5]], dtype=tf.float16)(cost)
    cost_vol = tf.nn.leaky_relu(cost_vol, alpha=0.1, name=name)

    return tf.cast(cost_vol, tf.float32)


x_in= ks.Input(shape=(96, 96, 32,), dtype=float32)  # data image
x = cost_volume(x_in, 4, cost_volume, 1, 2)
model = ks.Model(inputs=[x_in], outputs=[x])

x2 = cost_volume2(x_in, 4 , cost_volume, 1, 2)
model2 = ks.Model(inputs=[x_in], outputs=[x2])
# # model2.summary()


b=3
x_sample = tf.random.normal([b]+ x_in.shape[1:])
y1 = model(x_sample)
y2 = model2(x_sample)
print(y1.shape)
print(y2.shape)
# print(model(x_sample)[0,0,0,:])
# print(model2(x_sample)[0,0,0,:])

# print(y1[0,0,0,47:65])
# print(y2[0,0,0,47:65])
# print(tf.math.equal(y1, y2))
print(tf.reduce_all(tf.math.equal(y1, y2)))
